In [1]:
#import pandas as pd
import numpy as np
import logging
#import re
#import os
#import codecs


import tensorflow as tf
#import tensorflow_hub as hub
#from tensorflow_hub import resolver
#from keras import layers
#from keras.models import Model
#from bert import bert_tokenization

import storage
import preprocessing

In [ ]:
!echo $TFHUB_CACHE_DIR

BATCH_SIZE = 1000
MAX_CAPTION = 35
tf.__version__

In [ ]:
#We are using BERT_URL = https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1'

# Preprocessing

## The BERT layer requires 3 input sequence:

<li> <b>Token ids</b>: for every token in the sentence, gets Id from BERT vocab dictionary. </li>
<li> <b>Mask ids</b>: 1 for each token of the sentence, 0 otherwise. used for the sequence padding (so every sequence has the same length).</li>
<li> <b>Segment ids</b>: 0 for first sentence of the sequence, 1 for the second... etc...</li>


In [ ]:
# BERT Enbedding test
module = preprocessing.import_BERT_pretrained_model()
sentences = ['I prefer Python over Java', 'I like coding in Python', 'coding is fun']
preprocess_captions = preprocessing.preprocess_sentences(sentences)
pooled_output, sequence_output = preprocessing.BERT_embed(module, preprocess_captions)
print(pooled_output.shape)
print(sequence_output.shape)

In [ ]:
# Test of the Batch preparation

In [2]:
X, Y  = preprocessing.batch_generator("train", 0, 100)

In [3]:
X["image_input"].shape

(90, 300, 300, 3)

In [8]:
X["caption_input"].numpy().shape

(90, 35, 768)

In [9]:
Y["output"].numpy().shape

(90, 35, 768)